# SB125 Fund Split Analysis

## Question:
- How did RTAs split SB125 funds between operations and capital?

## Methodology:
- upload all avilable `SB125 fund request template` files to gcs
- examine all files for consistencies:
    - come with cleaning plan for inconsistent examples (files withot capital/operating columns)
- concat all rows across all files


## Notes:
- some RTPAs did not submit a `SB125 fund request template.xlsx` file, but instead included an quivilent file their allocation package

In [1]:
import pandas as pd
import os
import altair as alt

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

GCS_PATH = "gs://calitp-analytics-data/data-analyses/sb125/fund_split/"

In [2]:
file_list = [
    "sierra_fund_request.xlsx",
    "slocog_fund_request.xlsx",
    "tehema_fund_request.xlsx",
    "tuolumne_fund_request.xlsx",
    "ventura_fund_request.xlsx",
    "alpine_fund_request.xlsx",
    "amador_fund_request.xlsx",
    "butte_fund_request.xlsx",
    "calaveras_fund_request.xlsx",
    "del_norte_fund_request.xlsx",
    "el_dorado_fund_request.xlsx",
    "humboldt_fund_request.xlsx",
    "kern_fund_request.xlsx",
    "kings_fund_request.xlsx",
    "la_metro_fund_request.xlsx",
    "lake_fund_request.xlsx",
    "lassen_fund_request.xlsx",
    "madera_fund_request.xlsx",
    "mariposa_fund_request.xlsx",
    "mendocino_fund_request.xlsx",
    "merced_fund_request.xlsx",
    "mtc_fund_request.xlsx",
    "nevada_fund_request.xlsx",
    "orange_fund_request.xlsx",
    "placer_fund_request.xlsx",
    "plumas_fund_request.xlsx",
    "riverside_fund_request.xlsx",
    "san_benito_fund_request.xlsx",
    "san_diego_mts_fund_request.xlsx",
    "santa_cruz_fund_request.xlsx",
    "shasta_fund_request.xlsx",
]

file_list.sort()

In [3]:
def clean_fund_request(file:str) -> pd.DataFrame:
    """
    reads in the file from GCS, maps col_names list to df columns, drops all the blank rows.
    returns df.
    """
    col_names = [
    "rtpa",
    "implementing agenc-y/-ies",
    "project",
    "fund source",
    "capital_FY23-24",
    "capital_FY24-25",
    "capital_FY25-26",
    "capital_FY26-27",
    "operating_FY23-24",
    "operating_FY24-25",
    "operating_FY25-26",
    "operating_FY26-27",
    "total",
]
    
    df = pd.read_excel(f"{GCS_PATH}{file}", header=2, nrows=40, names=col_names).drop(columns="total")
    row_drop = df["rtpa"].isin(["Grand Total", "RTPA"])
    df = df.drop(df[row_drop].index)
    df = df.dropna(how= "all")
    df[["rtpa", "implementing agenc-y/-ies","project"]] = df[["rtpa", "implementing agenc-y/-ies","project"]].ffill()
    
    return df


first iteration of fund_request_checker func.
>def read_in(file:str) -> pd.DataFrame:
>    df = pd.read_excel(f"{GCS_PATH}{file}", nrows=40)
>    df = df.dropna(how= "all")
>   
>    if len(df.columns) == 13:
>        print(f"{file} can use clean_fund_request func.")
>    else: 
>        print(f"needs manual check, {file}")
       

old iteration
>def fund_request_checker(file_list:list) -> pd.DataFrame:
>
>    for file in file_list:
>    
>        df = pd.read_excel(f"{GCS_PATH}{file}", nrows=40)
>        df = df.dropna(how= "all")
>    
>        if len(df.columns) == 13:
>            print(f"{file} can use clean_fund_request func.")
>        else: 
>            print(f"needs manual check, {file}")
       

In [4]:
#fund_request_checker(file_list)

old iteration
>def fund_request_checker_v2(file_list:list):
>    gtg_files = []
>    manual_review = []
>    for file in file_list:
>    
>        df = pd.read_excel(f"{GCS_PATH}{file}", nrows=40)
>        df = df.dropna(how= "all")
>    
>        if len(df.columns) == 13:
>            gtg_files.append(f"{file}")
>        else: 
>            manual_review.append(f"{file}")
>    return display(
>        "good to go files",
>        list(gtg_files), 
>        "needs manual check", 
>        list(manual_review)
>    )

In [5]:
#fund_request_checker_v2(file_list)

In [6]:
def fund_request_checker_v3(file_list:list) -> tuple:
    """takes in list of fund_request excel file name. reads in each file, checks if DF has 13 columns.
    if yes, appends do good-to-go list. else, appends to needs-manual-review.
    output is a tuple of the 2 list.
    assign 2 variables to use this func.
    """
    gtg_files = []
    manual_review = []
    for file in file_list:
    
        df = pd.read_excel(f"{GCS_PATH}{file}", nrows=40)
        df = df.dropna(how= "all")
    
        if len(df.columns) == 13:
            gtg_files.append(f"{file}")
        else: 
            manual_review.append(f"{file}")
    return gtg_files, manual_review

In [7]:
good_list, review_list = fund_request_checker_v3(file_list)

In [8]:
display(
    len(good_list),
    len(review_list)
)

24

7

In [9]:
def cleaner_loop(gtg_list:list) -> dict: 
    """
    takes in good-to-go list from fund_request_checker.
    applies the clean_fund_request function to each item on the list, then appends to dictionary.
    key is name of the file, value is the cleaned dataframe.
    output is dictionary. 
    """
    cleaned_df ={}

    for name in gtg_list:
        cleaned_df[name] = clean_fund_request(name)
    return cleaned_df

In [10]:
cleaned_fund_request = cleaner_loop(good_list)

In [11]:
display(
    type(cleaned_fund_request),
    len(cleaned_fund_request),
    list(cleaned_fund_request.keys()),
)

dict

24

['alpine_fund_request.xlsx',
 'amador_fund_request.xlsx',
 'calaveras_fund_request.xlsx',
 'del_norte_fund_request.xlsx',
 'el_dorado_fund_request.xlsx',
 'humboldt_fund_request.xlsx',
 'kings_fund_request.xlsx',
 'la_metro_fund_request.xlsx',
 'lake_fund_request.xlsx',
 'madera_fund_request.xlsx',
 'mariposa_fund_request.xlsx',
 'mendocino_fund_request.xlsx',
 'merced_fund_request.xlsx',
 'nevada_fund_request.xlsx',
 'placer_fund_request.xlsx',
 'plumas_fund_request.xlsx',
 'riverside_fund_request.xlsx',
 'san_benito_fund_request.xlsx',
 'san_diego_mts_fund_request.xlsx',
 'shasta_fund_request.xlsx',
 'sierra_fund_request.xlsx',
 'slocog_fund_request.xlsx',
 'tehema_fund_request.xlsx',
 'tuolumne_fund_request.xlsx']

In [12]:
# view all the good-to-go df

from IPython.display import display

# .items() creates tuples of each element in the dict. key:value maps to key:df

for key, df in cleaned_fund_request.items():
    print(f"DataFrame: {key}")
    #display(df)


DataFrame: alpine_fund_request.xlsx
DataFrame: amador_fund_request.xlsx
DataFrame: calaveras_fund_request.xlsx
DataFrame: del_norte_fund_request.xlsx
DataFrame: el_dorado_fund_request.xlsx
DataFrame: humboldt_fund_request.xlsx
DataFrame: kings_fund_request.xlsx
DataFrame: la_metro_fund_request.xlsx
DataFrame: lake_fund_request.xlsx
DataFrame: madera_fund_request.xlsx
DataFrame: mariposa_fund_request.xlsx
DataFrame: mendocino_fund_request.xlsx
DataFrame: merced_fund_request.xlsx
DataFrame: nevada_fund_request.xlsx
DataFrame: placer_fund_request.xlsx
DataFrame: plumas_fund_request.xlsx
DataFrame: riverside_fund_request.xlsx
DataFrame: san_benito_fund_request.xlsx
DataFrame: san_diego_mts_fund_request.xlsx
DataFrame: shasta_fund_request.xlsx
DataFrame: sierra_fund_request.xlsx
DataFrame: slocog_fund_request.xlsx
DataFrame: tehema_fund_request.xlsx
DataFrame: tuolumne_fund_request.xlsx


# Cleaning individual good DFs

In [13]:
cleaned_fund_request["amador_fund_request.xlsx"] = cleaned_fund_request["amador_fund_request.xlsx"].drop([2])

In [14]:
cleaned_fund_request["merced_fund_request.xlsx"] = cleaned_fund_request["merced_fund_request.xlsx"].drop([1,2,34,36,37])

In [15]:
cleaned_fund_request["san_benito_fund_request.xlsx"] = cleaned_fund_request["san_benito_fund_request.xlsx"].drop([6,9])

In [16]:
row_drops = [
    1,
    2,
    3,
    4,
    9,
    10,
    11,
]
cleaned_fund_request["san_diego_mts_fund_request.xlsx"] = cleaned_fund_request["san_diego_mts_fund_request.xlsx"].drop(row_drops)

In [17]:
cleaned_fund_request["sierra_fund_request.xlsx"] = cleaned_fund_request["sierra_fund_request.xlsx"].drop(list(range(24,32)))

In [18]:
cleaned_fund_request["nevada_fund_request.xlsx"] = cleaned_fund_request["nevada_fund_request.xlsx"].drop([8,9])

In [19]:
cleaned_fund_request["plumas_fund_request.xlsx"]= cleaned_fund_request["plumas_fund_request.xlsx"].drop(list(range(10,14)))

# Attempt to clean bad DFs

In [20]:
    col_names = [
    "rtpa",
    "implementing agenc-y/-ies",
    "project",
    "fund source",
    "capital_FY23-24",
    "capital_FY24-25",
    "capital_FY25-26",
    "capital_FY26-27",
    "operating_FY23-24",
    "operating_FY24-25",
    "operating_FY25-26",
    "operating_FY26-27",
    "total",
    ]

In [21]:
# list of bad DFs that didnt match the other layouts
review_list

['butte_fund_request.xlsx',
 'kern_fund_request.xlsx',
 'lassen_fund_request.xlsx',
 'mtc_fund_request.xlsx',
 'orange_fund_request.xlsx',
 'santa_cruz_fund_request.xlsx',
 'ventura_fund_request.xlsx']

In [22]:
#CLEAN COMPLETED

lassen = pd.read_excel(f"{GCS_PATH}lassen_fund_request.xlsx", 
                       sheet_name = "D.2. Detailed Fund Request",
                       skiprows = 6,
                       header=0, 
                       skipfooter=12,
                      ).drop(columns=["Unnamed: 0", "Project Type", "Operator"])

# can work with this. may be able to remove the top rows then use cleaner loop
lassen.columns=col_names
lassen_cleaned = lassen.drop(columns="total")

In [23]:
#lassen_cleaned

In [24]:
# CLEANED COMPLETE
butte = pd.read_excel(f"{GCS_PATH}butte_fund_request.xlsx",
                      skiprows= 2,
                      header= 0,
                      skipfooter= 17,
                     )

butte_cleaned = butte.copy()

butte_cleaned[["RTPA","Implementing Agenc-y/-ies", "Project"]] = butte_cleaned[["RTPA","Implementing Agenc-y/-ies", "Project"]].ffill()

butte_cleaned.insert(6, "operations FY25-26",0)
butte_cleaned.insert(7, "operations FY26-27",0)
butte_cleaned.columns = [
    "rtpa",
    "implementing agenc-y/-ies",
    "project",
    "fund source",
    "operating_FY23-24",
    "operating_FY24-25",
    "operating_FY25-26",
    "operating_FY26-27",
    "capital_FY23-24",
    "capital_FY24-25",
    "capital_FY25-26",
    "capital_FY26-27",
    "total",
    
]
butte_cleaned = butte_cleaned.drop(columns="total")
# missing the operations columns. may not need if we melt just the capital columns
display(
    butte.shape,
    butte_cleaned.shape,
    #butte_cleaned,
)

(5, 11)

(5, 12)

In [25]:
# CLEANED COMPLETE

mtc = pd.read_excel(f"{GCS_PATH}mtc_fund_request.xlsx",
                    skiprows = 2,
                    header= 0,
                    skipfooter= 21
                   ).drop(columns= ["Unnamed: 13","Unnamed: 14","Unnamed: 15"])

mtc_cleaned = mtc.copy()

mtc_cleaned.columns = col_names
mtc_cleaned = mtc_cleaned.drop(columns="total")
# can work with this. may be able to remove the top rows then use cleaner loop
display(
    mtc_cleaned.shape,
    mtc_cleaned.columns,
    #mtc_cleaned
)

(16, 12)

Index(['rtpa', 'implementing agenc-y/-ies', 'project', 'fund source',
       'capital_FY23-24', 'capital_FY24-25', 'capital_FY25-26',
       'capital_FY26-27', 'operating_FY23-24', 'operating_FY24-25',
       'operating_FY25-26', 'operating_FY26-27'],
      dtype='object')

In [26]:
#CLEANED COMPLETE 
orange = pd.read_excel(f"{GCS_PATH}orange_fund_request.xlsx",
                       skiprows=3,
                       header=0,
                       skipfooter=1
                      )

orange_cleaned = orange.copy()

orange_cleaned.rename(columns={"Unnamed: 0":"RTPA"}, inplace = True)
orange_cleaned["RTPA"] = "OCTA"
orange_cleaned = orange_cleaned.drop(columns= [
    "FY27-28",
    "FY28-29",
    "FY27-28.1",
    "FY28-29.1"
])

orange_cleaned.columns = col_names
orange_cleaned = orange_cleaned.drop(columns="total")
# FY goes to 28-29. dont think i need those columns, there are no numbers in them anyways
# can drop
#orange_cleaned

In [27]:
# CLEANED COMPLETE
santa_cruz = pd.read_excel(f"{GCS_PATH}santa_cruz_fund_request.xlsx",
                           skiprows= 4,
                           header= 0,
                           skipfooter= 5
                          ).iloc[:,0:13]

santa_cruz_cleaned = santa_cruz.copy()

santa_cruz_cleaned.columns = col_names
santa_cruz_cleaned.drop(columns="total", inplace=True)
# can remove the extra columns past unnamed 12, then remove top and bottom rows, then use cleaner loop
display(
    santa_cruz_cleaned.shape,
    #santa_cruz_cleaned
)

(8, 12)

In [56]:
ventura_tircp_capital = pd.read_excel(f"{GCS_PATH}ventura_fund_request.xlsx", 
                                      sheet_name = "Project Breakdown",
                                      skiprows = 2,
                                      header = 0,
                                      skipfooter = 40
                                     )

# needs a lot of work, may have to do in excel
# try breaking up the DF into multiple seperate DFs, then add them all together at the end
# will have to add capital or operations columns as needed. 
ventura_tircp_capital

,Implementing Agenc-y/-ies,Project Category,Project,Year Requested,FY23-24,FY24-25,FY25-26,FY26-27,Unnamed: 8
0,Gold Coast Transit District,"Infrastructure (zero emission, upgrades, bus s...",Energy Storage - Hydrogen Station,Year 1,NaN,1124000.0,33543.0,NaN,NaN
1,Simi Valley,"Infrastructure (zero emission, upgrades, bus s...",EV Charging,Year 1,NaN,651953.0,NaN,NaN,NaN
2,County of Ventura,"Infrastructure (zero emission, upgrades, bus s...",Bus Stop Improvements,Year 1,NaN,NaN,1000.0,NaN,NaN
3,Thousand Oaks,"Infrastructure (zero emission, upgrades, bus s...",EV Charging,Year 1,NaN,1000000.0,NaN,NaN,NaN
4,Thousand Oaks,"Infrastructure (zero emission, upgrades, bus s...",EV Support,Year 1,NaN,500000.0,NaN,NaN,NaN
5,Gold Coast Transit District,"Infrastructure (zero emission, upgrades, bus s...",Energy Storage - Hydrogen Station,Year 2,NaN,NaN,468504.0,NaN,NaN
6,Valley Express,"Infrastructure (zero emission, upgrades, bus s...",Bus Stop Improvements,Year 2,NaN,NaN,50000.0,NaN,NaN
7,Ojai,"Infrastructure (zero emission, upgrades, bus s...",Facility Upgrades & EV Charging,Year 1,NaN,15000.0,NaN,NaN,NaN
8,Ojai,"Infrastructure (zero emission, upgrades, bus s...",Facility Upgrades & EV Charging,Year 2,NaN,200000.0,NaN,NaN,NaN
9,Ventura County Transporation Commission,Technology,New Fare System,Year 1,NaN,4000000.0,NaN,NaN,NaN


In [61]:
ventura_zetcp_capital = pd.read_excel(f"{GCS_PATH}ventura_fund_request.xlsx", 
                                      sheet_name = "Project Breakdown",
                                      skiprows = 32,
                                      header = 0,
                                      skipfooter = 21
                                     )
ventura_zetcp_capital

,Implementing Agenc-y/-ies,Project,Fund Source,FY23-24,FY24-25,FY25-26,FY26-27,Unnamed: 7,Unnamed: 8
0,Simi Valley,EV Charging,PTA,NaN,2348047.0,NaN,NaN,NaN,NaN
1,Gold Coast Transit District,Hydrogen Station - Energy Storage,GGRF Y1,NaN,666953.0,NaN,NaN,NaN,NaN
2,Gold Coast Transit District,Relief vehicles,GGRF Y1,NaN,51998.0,NaN,NaN,NaN,NaN
3,Moorpark,EV Charging,GGRF Y1,NaN,200000.0,NaN,NaN,NaN,NaN
4,Thousand Oaks,DAR vehicle replacement,GGRF Y1,NaN,1800000.0,NaN,NaN,NaN,NaN
5,Gold Coast Transit District,Relief vehicles,GGRF Y2,NaN,42463.0,NaN,NaN,NaN,NaN
6,Camarillo,EV Charging,GGRF Y2,NaN,NaN,500000.0,NaN,NaN,NaN
7,Thousand Oaks,DAR vehicle replacement,GGRF Y2,NaN,NaN,1800000.0,NaN,NaN,NaN
8,VCTC Intercity - VCTC,EV Charging,GGRF Y2,NaN,NaN,500000.0,NaN,NaN,NaN
9,VCTC Intercity - VCTC,EV Charging,GGRF Y3,NaN,NaN,NaN,1342463.0,NaN,NaN


In [68]:
ventura_tircp_operating = pd.read_excel(f"{GCS_PATH}ventura_fund_request.xlsx", 
                                      sheet_name = "Project Breakdown",
                                      skiprows = 51,
                                      header = 0,
                                      skipfooter = 1
                                     )
ventura_tircp_operating

,Implementing Agenc-y/-ies,Project Category,Project,Fund Source,FY23-24,FY24-25,FY25-26,FY26-27,Unnamed: 8
0,Gold Coast Transit District,Operations Support,Operations Support,TIRCP Y1,NaN,4871741.0,7790198.0,NaN,NaN
1,Valley Express,Operations Support,Operations Support,TIRCP Y1,400000.0,NaN,NaN,NaN,NaN
2,County of Ventura,Operations Support,Operations Support,TIRCP Y1,NaN,31350.0,NaN,NaN,NaN
3,Ojai,Operations Support,Operations Support,TIRCP Y1,NaN,30000.0,NaN,NaN,NaN
4,Simi Valley,Operations Support,Microtransit Program,TIRCP Y1,NaN,1500000.0,150000.0,NaN,NaN
5,Ventura County Transporation Commission,Operations Support,Radio System,TIRCP Y1,NaN,65000.0,NaN,NaN,NaN
6,Ventura County Transporation Commission,Operations Support,Free Fare Programs,TIRCP Y1,350000.0,1100000.0,NaN,NaN,NaN
7,Ventura County Transporation Commission,Operations Support,Admin support,TIRCP Y1,30000.0,NaN,NaN,NaN,NaN
8,Gold Coast Transit District,Operations Support,Operations Support,TIRCP Y2,NaN,NaN,NaN,9520904.0,NaN
9,Gold Coast Transit District,Operations Support,Matching Grant Funds,TIRCP Y2,NaN,1643995.0,NaN,NaN,NaN


In [29]:
kern = pd.read_excel(f"{GCS_PATH}kern_fund_request.xlsx")

# this is so bad, may need to do this in excel to make it work. 
# the agency name and fund soruce are on the same column
kern

,Summary Excel table of proposed TIRCP and ZETCP funds,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8
0,NaN,FY 2023-24,NaN,FY 2024-25,NaN,FY 2025-26,NaN,FY 2026-27,NaN
1,Agency,Capital,Operational,Capital,Operational,Capital,Operational,Capital,Operatioanl
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Arvin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,TIRCP,1108563,0,1118595,0,0,0,0,0
5,ZETCP,120575,0,68316,0,68316,0,68316,0
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,California City,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,TIRCP,711044,0,717478,0,0,0,0,0
9,ZETCP,77338,0,43819,0,43819,0,43819,0


# Test of concat all the dictionary dataframe vales

In [30]:
fund_request_df = cleaned_fund_request.values()

type(fund_request_df)

dict_values

In [31]:
all_fund_request = pd.concat(cleaned_fund_request.values(), ignore_index=True)

In [32]:
all_fund_request = pd.concat([all_fund_request,
                              santa_cruz_cleaned,
                              orange_cleaned,
                              butte_cleaned,
                              lassen_cleaned
                             ], ignore_index=True)

In [33]:
display(
    all_fund_request.shape,
    type(all_fund_request),
    all_fund_request.columns
)

(210, 12)

pandas.core.frame.DataFrame

Index(['rtpa', 'implementing agenc-y/-ies', 'project', 'fund source',
       'capital_FY23-24', 'capital_FY24-25', 'capital_FY25-26',
       'capital_FY26-27', 'operating_FY23-24', 'operating_FY24-25',
       'operating_FY25-26', 'operating_FY26-27'],
      dtype='object')

In [34]:
# sum all the "capital columns"
#all_fund_request[["capital_FY23-24":"capital_FY26-27"]].sum()

In [35]:
#sum all the "operations columns"

# TEST of Melting the dataframe

In [36]:
id_vars= [
    'rtpa',
 'implementing agenc-y/-ies',
 'project',
 'fund source',
]
val_vars = [
    'capital_FY23-24',
 'capital_FY24-25',
 'capital_FY25-26',
 'capital_FY26-27',
 'operating_FY23-24',
 'operating_FY24-25',
 'operating_FY25-26',
 'operating_FY26-27'
]

melt = all_fund_request.melt(
    id_vars = id_vars,
    value_vars = val_vars,
    var_name = "capital/operation fy",
    value_name = "fund amount",
    ignore_index = True)

In [37]:
display(
    type(melt),
    melt.shape,
    melt.dtypes,
    melt.head()
)

pandas.core.frame.DataFrame

(1680, 6)

rtpa                         object
implementing agenc-y/-ies    object
project                      object
fund source                  object
capital/operation fy         object
fund amount                  object
dtype: object

,rtpa,implementing agenc-y/-ies,project,fund source,capital/operation fy,fund amount
0,Alpine County Transportation Commission,Alpine County Transportation Commission,Transit Facility Conversion Project,TIRCP,capital_FY23-24,360641.0
1,Alpine County Transportation Commission,Alpine County Transportation Commission,Transit Facility Conversion Project,TIRCP,capital_FY23-24,NaN
2,Alpine County Transportation Commission,Alpine County Transportation Commission,Transit Facility Conversion Project,ZETCP (GGRF),capital_FY23-24,3616.684
3,Alpine County Transportation Commission,Alpine County Transportation Commission,Transit Facility Conversion Project,ZETCP (PTA),capital_FY23-24,3123.316
4,Amador County Transportation Commission,Amador Transit,NaN,TIRCP,capital_FY23-24,100000.0


In [38]:
# splitting the cap/operations columns

melt[["project type", "fiscal year"]] = melt["capital/operation fy"].str.split('_FY', expand = True)

In [39]:
# updates the `-` string to zero.
melt.loc[(melt["rtpa"] == "Humboldt County Association of Governments") & (melt["fund amount"] == "-"),"fund amount"] = 0

In [40]:
# check for NaNs
melt["fund amount"].isna().sum()

1248

In [41]:
melt["fund amount"] = melt["fund amount"].fillna(0).astype("int64")
melt["fund source"] = melt["fund source"].astype("str")

In [42]:
# ensure no more NaNs
melt["fund amount"].isna().sum()

0

In [43]:
display(
    melt.columns,
    melt.sample(3)
)

Index(['rtpa', 'implementing agenc-y/-ies', 'project', 'fund source',
       'capital/operation fy', 'fund amount', 'project type', 'fiscal year'],
      dtype='object')

,rtpa,implementing agenc-y/-ies,project,fund source,capital/operation fy,fund amount,project type,fiscal year
1556,Placer County Transportation Planning Agency (...,Capitol Corridor Joint Powers Authority,Sacramento to Roseville Third Main Track: Phas...,TIRCP,operating_FY26-27,0,operating,26-27
1050,Alpine County Transportation Commission,Alpine County Transportation Commission,Transit Facility Conversion Project,TIRCP,operating_FY24-25,0,operating,24-25
1317,Mendocino Council of Governments,Mendocino Council of Governments,Administration & Planning,ZETCP (PTA),operating_FY25-26,0,operating,25-26


In [44]:
# checking fund amounts for any non-int values 
melt["fund amount"].unique()

array([   360641,         0,      3616,      3123,    100000,   2576611,
          238532,    500000,   1175501,     69395,     80357,   9085857,
          242312,    521458,   6849293,    329561,    295818,   1133408,
          101796, 498650905, 119494973,   3706510,     87574,    198860,
           84159,   8254231,    108201,     42539,     49259,   4848229,
          123518,    271142,    110636,    104561,    696393,    722403,
          836515,   1000000,   2080000,   6400000,    200000,    370000,
         1486685,    297576,    133646,  10000000,    400000,     92109,
          501942,   1019544,   3354086,   2510740,    614200,    250000,
          100520,     61421,     53042,  63382700,   4003053,   7956643,
         2868594,  60000000,   2328990,    348002,  10126000,   5434000,
         1705263,  26000000,    943316,   2050000,      9408,      8124,
          124000,    280000,    395000,   1778000,   1400000,   4000000,
          387000,    375000,    600000,    422000, 

In [45]:
display(
    melt.dtypes,
    melt["fund source"].value_counts()
)

rtpa                         object
implementing agenc-y/-ies    object
project                      object
fund source                  object
capital/operation fy         object
fund amount                   int64
project type                 object
fiscal year                  object
dtype: object

TIRCP           936
ZETCP (GGRF)    352
ZETCP (PTA)     296
ZETCP            40
CMAQ             16
Measure V         8
5339              8
SGR               8
Farebox           8
5307              8
Name: fund source, dtype: int64

# Draft Aggregations

In [46]:
def make_bar(data, x_axis, y_axis):
    chart = alt.Chart(data).mark_bar().encode(
        x = x_axis,
        y = y_axis,
    )
    return chart

In [47]:
by_type = melt.groupby(["project type"]).agg({
    "fund amount": "sum",
}).reset_index()

make_bar(by_type, y_axis = "project type", x_axis = "fund amount")

alt.Chart(...)

In [48]:
by_year = melt.groupby(["fiscal year","project type"]).agg({
    "fund amount": "sum",
}).reset_index()

make_bar(by_year, y_axis = "fiscal year", x_axis = "fund amount",)

alt.Chart(...)

In [49]:
by_source = melt.groupby(["fund source"]).agg({
    "fund amount": "sum",
        "rtpa": "nunique"
}).reset_index()

make_bar(by_source, y_axis = "fund source", x_axis = "fund amount")

alt.Chart(...)

In [50]:
by_rtpa = melt.groupby(["rtpa"]).agg({
    "fund amount": "sum",
    "project": "nunique"
}).reset_index()

make_bar(by_rtpa, "rtpa", "fund amount")

alt.Chart(...)